In [2]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

In [3]:
clist1 =['#7cb3d4','#669dc7','#5185bb','#426cae','#3951a0','#303692',
        '#a60025','#bb1624','#d12927','#e14230','#ed5c3d','#f6774b',
        '#fa9359','#fdaf67','#fdc77c','#ffdc8f','#feeaa5','#fff8b7',
        '#f8fccd','#ebf7e4','#dcf1f7','#c4e6f0','#aadae8','#94c6de',]
red1 = LinearSegmentedColormap.from_list('chaos',clist1,N=24)
dpi_ = 100

In [4]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')

In [38]:
ds_t_dtmean

<xarray.Dataset> Size: 13MB
Dimensions:  (lat: 707, lon: 1137)
Coordinates:
  * lon      (lon) float64 9kB -124.8 -124.8 -124.7 ... -66.99 -66.94 -66.89
  * lat      (lat) float64 6kB 24.4 24.43 24.47 24.5 ... 49.24 49.28 49.31 49.35
    XLON     (lat, lon) float32 3MB -118.9 -118.9 -118.9 ... -65.61 -65.56
    XLAT     (lat, lon) float32 3MB 23.44 23.45 23.46 23.47 ... 47.4 47.39 47.38
Data variables:
    t        (lat, lon) float64 6MB ...

In [6]:
ds_t_dtmean = xr.open_dataset('slope_linereg/tslope_dtmean.nc')
ds_t_dtmin = xr.open_dataset('slope_linereg/tslope_dtmin.nc')
ds_t_ntmean = xr.open_dataset('slope_linereg/tslope_ntmean.nc')
ds_t_ntmin = xr.open_dataset('slope_linereg/tslope_ntmin.nc')

ds_dt_dtmean = xr.open_dataset('slope_linereg/dtslope_dtmean.nc')
ds_dt_dtmean = xr.open_dataset('slope_linereg/dtslope_dtmin.nc')
ds_dt_ntmean = xr.open_dataset('slope_linereg/dtslope_ntmean.nc')
ds_dt_ntmean = xr.open_dataset('slope_linereg/dtslope_ntmin.nc')

ds_p_dtint = xr.open_dataset('slope_linereg/pslope_dtint.nc')
ds_p_dtfreq = xr.open_dataset('slope_linereg/pslope_dtfreq.nc')
ds_p_ntint = xr.open_dataset('slope_linereg/pslope_ntint.nc')
ds_p_ntfreq = xr.open_dataset('slope_linereg/pslope_ntfreq.nc')

In [7]:
ds_raster = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2022-09-30_23:00:00.nc')

In [8]:
lon = ds_raster['XLONG'].values
lat = ds_raster['XLAT'].values
grid = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
    index=np.arange(lon.size)
)
grid.set_crs(gdf.crs, inplace=True)
grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

mask = np.full(ds_raster['PREC_ACC_NC'].shape[1:], False) 
for index in grid_s.index:
    row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
    mask[row, col] = True
mask_da = xr.DataArray(mask, dims=ds_raster['PREC_ACC_NC'].dims[1:], coords={'south_north': ds_raster['PREC_ACC_NC'].coords['south_north'], 'west_east': ds_raster['PREC_ACC_NC'].coords['west_east']})
ds_conus = ds_raster.where(mask_da, drop=True)

In [9]:
XLON = ds_conus.XLONG.values[:707,:]
XLAT = ds_conus.XLAT.values[:707,:]
ds_t_dtmean = ds_t_dtmean.assign_coords({
    'XLON': (('lat', 'lon'), XLON),
    'XLAT': (('lat', 'lon'), XLAT)
})

In [10]:
regions_dict = {
    'NE': ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'WV'],
    'Midwest': ['IA', 'MI', 'MN', 'WI', 'IL', 'IN', 'MO', 'OH'],
    'SE': ['AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 'KY', 'AR', 'LA', 'MS'],
    'NGP': ['MT', 'NE', 'ND', 'SD', 'WY'],
    'SGP': ['KS', 'OK', 'TX'],
    'SW': ['AZ', 'CO', 'NM', 'UT', 'CA', 'NV'],
    'NW': ['ID', 'OR', 'WA']
}
regions = {name: gdf[gdf['STUSPS'].isin(states)] for name, states in regions_dict.items()}

In [11]:
regi = ['NE','Midwest','SE','NGP','SGP','SW','NW',]

In [12]:


# 文件路径和变量名对应的字典
file_paths = {
    't_dtmean': 'slope_linereg/tslope_dtmean.nc',
    't_dtmin': 'slope_linereg/tslope_dtmin.nc',
    't_ntmean': 'slope_linereg/tslope_ntmean.nc',
    't_ntmin': 'slope_linereg/tslope_ntmin.nc',
    'dt_dtmean': 'slope_linereg/dtslope_dtmean.nc',
    'dt_dtmin': 'slope_linereg/dtslope_dtmin.nc',
    'dt_ntmean': 'slope_linereg/dtslope_ntmean.nc',
    'dt_ntmin': 'slope_linereg/dtslope_ntmin.nc',
}

# 读取所有文件并存储在字典中
datasets = {key: xr.open_dataset(path) for key, path in file_paths.items()}

# 处理每个区域
ds_results_t = {}

for key, ds in datasets.items():
    # 添加坐标
    ds = ds.assign_coords({
        'XLON': (('lat', 'lon'), ds_conus.XLONG.values[:707, :]),
        'XLAT': (('lat', 'lon'), ds_conus.XLAT.values[:707, :]),
    })
    
    # 按区域处理数据
    for r in regi:
        lon = ds['XLON'].values
        lat = ds['XLAT'].values
        grid = gpd.GeoDataFrame(
            geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
            index=np.arange(lon.size)
        )

        grid.set_crs(regions[r].crs, inplace=True)
        grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')

        # 创建布尔掩码
        mask = np.full((ds['t'].shape[0], ds['t'].shape[1]), False)
        for index in grid_s.index:
            row, col = np.unravel_index(index, mask.shape)
            mask[row, col] = True

        mask_da = xr.DataArray(
            mask,
            dims=['lat', 'lon'],
            coords={
                'lat': ds['lat'].values,
                'lon': ds['lon'].values
            }
        )

        # 筛选区域数据
        ds_ssss = ds.where(mask_da, drop=True)
        ds_results_t[f'{key}_{r}'] = ds_ssss


In [13]:


# 文件路径列表
file_paths = {
    'p_dtint': 'slope_linereg/pslope_dtint.nc',
    'p_dtfreq': 'slope_linereg/pslope_dtfreq.nc',
    'p_ntint': 'slope_linereg/pslope_ntint.nc',
    'p_ntfreq': 'slope_linereg/pslope_ntfreq.nc',
}

# 读取数据集
datasets = {key: xr.open_dataset(path) for key, path in file_paths.items()}

# 初始化结果字典
ds_results_p = {}

# 处理每个数据集
for key, ds in datasets.items():
    # 添加坐标
    ds = ds.assign_coords({
        'XLON': (('lat', 'lon'), ds_conus.XLONG.values[:707, :]),
        'XLAT': (('lat', 'lon'), ds_conus.XLAT.values[:707, :]),
    })
    
    # 针对每个区域进行处理
    for r in regi:
        lon = ds['XLON'].values
        lat = ds['XLAT'].values
        grid = gpd.GeoDataFrame(
            geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
            index=np.arange(lon.size)
        )
        
        # 设置坐标系
        grid.set_crs(regions[r].crs, inplace=True)
        grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')
        
        # 创建布尔掩码
        mask = np.full((ds['p'].shape[0], ds['p'].shape[1]), False)
        for index in grid_s.index:
            row, col = np.unravel_index(index, mask.shape)
            mask[row, col] = True
        
        # 转换为 xarray.DataArray
        mask_da = xr.DataArray(
            mask,
            dims=['lat', 'lon'],
            coords={
                'lat': ds['lat'].values,
                'lon': ds['lon'].values
            }
        )
        
        # 筛选区域数据
        ds_ssss = ds.where(mask_da, drop=True)
        ds_results_p[f'{key}_{r}'] = ds_ssss


In [19]:
dic_tmean_dt = {key: ds_results_t[key] for key in ['t_dtmean_NE', 't_dtmean_Midwest', 't_dtmean_SE', 't_dtmean_NGP', 't_dtmean_SGP', 't_dtmean_SW', 't_dtmean_NW']}
dic_tmin_dt = {key: ds_results_t[key] for key in ['t_dtmin_NE', 't_dtmin_Midwest', 't_dtmin_SE', 't_dtmin_NGP', 't_dtmin_SGP', 't_dtmin_SW', 't_dtmin_NW']}
dic_tmean_nt = {key: ds_results_t[key] for key in ['t_ntmean_NE', 't_ntmean_Midwest', 't_ntmean_SE', 't_ntmean_NGP', 't_ntmean_SGP', 't_ntmean_SW', 't_ntmean_NW']}
dic_tmin_nt = {key: ds_results_t[key] for key in ['t_ntmin_NE', 't_ntmin_Midwest', 't_ntmin_SE', 't_ntmin_NGP', 't_ntmin_SGP', 't_ntmin_SW', 't_ntmin_NW']}

dic_dtmean_dt = {key: ds_results_t[key] for key in ['dt_dtmean_NE', 'dt_dtmean_Midwest', 'dt_dtmean_SE', 'dt_dtmean_NGP', 'dt_dtmean_SGP', 'dt_dtmean_SW', 'dt_dtmean_NW']}
dic_dtmin_dt = {key: ds_results_t[key] for key in ['dt_dtmin_NE', 'dt_dtmin_Midwest', 'dt_dtmin_SE', 'dt_dtmin_NGP', 'dt_dtmin_SGP', 'dt_dtmin_SW', 'dt_dtmin_NW']}
dic_dtmean_nt = {key: ds_results_t[key] for key in ['dt_ntmean_NE', 'dt_ntmean_Midwest', 'dt_ntmean_SE', 'dt_ntmean_NGP', 'dt_ntmean_SGP', 'dt_ntmean_SW', 'dt_ntmean_NW']}
dic_dtmin_nt = {key: ds_results_t[key] for key in ['dt_ntmin_NE', 'dt_ntmin_Midwest', 'dt_ntmin_SE', 'dt_ntmin_NGP', 'dt_ntmin_SGP', 'dt_ntmin_SW', 'dt_ntmin_NW']}

dic_pint_dt = {key: ds_results_p[key] for key in ['p_dtint_NE', 'p_dtint_Midwest', 'p_dtint_SE', 'p_dtint_NGP', 'p_dtint_SGP', 'p_dtint_SW', 'p_dtint_NW']}
dic_pint_nt = {key: ds_results_p[key] for key in ['p_ntint_NE', 'p_ntint_Midwest', 'p_ntint_SE', 'p_ntint_NGP', 'p_ntint_SGP', 'p_ntint_SW', 'p_ntint_NW']}
dic_pfreq_dt = {key: ds_results_p[key] for key in ['p_dtfreq_NE', 'p_dtfreq_Midwest', 'p_dtfreq_SE', 'p_dtfreq_NGP', 'p_dtfreq_SGP', 'p_dtfreq_SW', 'p_dtfreq_NW']}
dic_pfreq_nt = {key: ds_results_p[key] for key in ['p_ntfreq_NE', 'p_ntfreq_Midwest', 'p_ntfreq_SE', 'p_ntfreq_NGP', 'p_ntfreq_SGP', 'p_ntfreq_SW', 'p_ntfreq_NW']}

In [32]:
dic_pint_dt['p_dtint_NE'].p.values.shape

(338, 265)

In [26]:
dic_tmean_dt['t_dtmean_NE'].t.values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [31]:
array = dic_pint_dt['p_dtint_NE'].p.values

# 计算非nan的数量
non_nan_count = np.count_nonzero(~np.isnan(array))

print(non_nan_count)

34639


In [36]:
dic_pint_dt['p_dtint_SE'].p.values.shape

(426, 426)

In [37]:
426*426

181476